### 待做思路：
1.画训练和测试集的特征值分布图，观察是否存在分布差异较大的特征，做剔除。

In [1]:
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from gensim.models import Word2Vec
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
data = pd.concat([train, test], axis=0, ignore_index=True)

In [7]:
#平均丢弃数据呼叫数,平均占线数据调用次数,长尾，大于10设为11
# data = data.apply(lambda x:11 if x["平均丢弃数据呼叫数"]>10)
# data['平均占线数据调用次数'] = data['平均占线数据调用次数'].apply(lambda x:11 if x>10)
# data['平均客户服务电话次数'] = data['平均客户服务电话次数'].apply(lambda x:51 if x>50)

SyntaxError: invalid syntax (<ipython-input-7-fe77d1e9ee6a>, line 2)

In [8]:
features = [f for f in data.columns if f not in ['是否流失','客户ID']]

train = data[data['是否流失'].notnull()].reset_index(drop=True)
test = data[data['是否流失'].isnull()].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['是否流失']

In [16]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2022
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []
    test_pred_sum = np.zeros(len(test))

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************{}***{} ************************************'.format(clf_name,str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.7,
                'bagging_fraction': 0.7,
                'bagging_freq': 10,
                'learning_rate': 0.2,
                'seed': 2022,
                'n_jobs':-1
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=3000, early_stopping_rounds=400)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            #print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 7,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.2,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 36
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=3000, early_stopping_rounds=400)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
                 
        if clf_name == "cat":
            params = {'learning_rate': 0.2, 'depth': 5, 'l2_leaf_reg': 10, 'bootstrap_type': 'Bernoulli',
                      'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False}
            
            model = clf(iterations=20000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      cat_features=[], use_best_model=True, verbose=3000)
            
            val_pred  = model.predict(val_x)
            test_pred = model.predict(test_x)
        
        test_pred_sum += test_pred
        #计算当前折验证集auc
        #train[valid_index] = np.argmax(val_pred)
        cv_scores.append(roc_auc_score(val_y, val_pred))
        print("cur fold auc:{}".format(roc_auc_score(val_y, val_pred)))
        print("test_pred:{}".format(test_pred))
       
    test = test_pred_sum/folds
    print(test)  
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    #print("%s_score_std:" % clf_name, np.std(cv_scores))
    return _,test
    
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test

def xgb_model(x_train, y_train, x_test):
    xgb_train, xgb_test = cv_model(xgb, x_train, y_train, x_test, "xgb")
    return xgb_train, xgb_test

def cat_model(x_train, y_train, x_test):
    cat_train, cat_test = cv_model(CatBoostRegressor, x_train, y_train, x_test, "cat") 
    return cat_train, cat_test
    


In [17]:
lgb_train, lgb_test = lgb_model(x_train, y_train, x_test)
xgb_train, xgb_test = xgb_model(x_train, y_train, x_test)
#cat_train, cat_test = cat_model(x_train, y_train, x_test)
final_test = (lgb_test + xgb_test) / 2

************************************lgb***1 ************************************
[LightGBM] [Info] Number of positive: 60034, number of negative: 59966
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021865 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10526
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 67
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500283 -> initscore=0.001133
[LightGBM] [Info] Start training from score 0.001133
Training until validation scores don't improve for 400 rounds
[3000]	training's auc: 0.999559	valid_1's auc: 0.81226
[6000]	training's auc: 1	valid_1's auc: 0.83275
[9000]	training's auc: 1	valid_1's auc: 0.840364
[12000]	training's auc: 1	valid_1's auc: 0.843659
[15000]	training's auc: 1	valid_1's auc: 0.844899
Early stopping, best iteration is:
[16920]	training's auc: 1	valid_1's auc: 0.845641
cur fold auc:0.84564122

[0]	train-auc:0.64536	eval-auc:0.63250
[3000]	train-auc:1.00084	eval-auc:0.82445
[6000]	train-auc:1.00084	eval-auc:0.82785
[9000]	train-auc:1.00084	eval-auc:0.82875
[10667]	train-auc:1.00084	eval-auc:0.82902
cur fold auc:0.8290604206839737
test_pred:[0.41751552 0.2733318  0.7923149  ... 0.07118481 0.05441161 0.48694357]
************************************xgb***5 ************************************
[23:54:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.64663	eval-auc:0.63727
[3000]	train-auc:1.00084	eval-auc:0.81800
[6000]	train-auc:1.00084	eval-auc:0.82114
[7726]	train-auc:1.00084	eval-auc:0.82161
cur fold auc:0.82

In [18]:
lgb_test,xgb_test

(array([0.30665911, 0.56230849, 0.58889371, ..., 0.00366054, 0.12776691,
        0.32711678]),
 array([0.2900366 , 0.5740195 , 0.65713514, ..., 0.04331771, 0.12369604,
        0.40267312]))

In [19]:
test['是否流失'] = final_test
test[['客户ID','是否流失']].to_csv('./data/test_sub.csv', index=False)

In [27]:
test['是否流失'] = lgb_test
test[['客户ID','是否流失']].to_csv('./data/lgb_test_sub.csv', index=False)

In [16]:
test['是否流失'] = xgb_test
test[['客户ID','是否流失']].to_csv('./data/xgb_test_sub.csv', index=False)

In [17]:
test['是否流失'] = cat_test
test[['客户ID','是否流失']].to_csv('./data/cat_test_sub.csv', index=False)